# Goal:

- Construct a graph in DGL from scratch.

- Assign node and edge features to a graph.

- Query properties of a DGL graph such as node degrees and connectivity.

- Transform a DGL graph into another graph.

- Load and save DGL graphs.

You can construct a graph by specifying the number of nodes in the graph as well as the list of source and destination nodes. Nodes in the graph have consecutive IDs starting from 0.

In [2]:
import dgl
import numpy as np
import torch

g = dgl.graph(([0, 0, 0, 0, 0], [1, 2, 3, 4, 5]), num_nodes=6)
# Equivalently, PyTorch LongTensors also work.
g = dgl.graph((torch.LongTensor([0, 0, 0, 0, 0]), torch.LongTensor([1, 2, 3, 4, 5])), num_nodes=6)

# You can omit the number of nodes argument if you can tell the number of nodes from the edge list alone.
g = dgl.graph(([0, 0, 0, 0, 0], [1, 2, 3, 4, 5]))

Edges in the graph have consecutive IDs starting from 0, and are in the same order as the list of source and destination nodes during creation.

In [3]:
# Print the source and destination nodes of every edge.
print(g.edges())

(tensor([0, 0, 0, 0, 0]), tensor([1, 2, 3, 4, 5]))


# Assigning Node and Edge Features to Graph

Many graph data contain attributes on nodes and edges. Although the types of node and edge attributes can be arbitrary in real world, DGLGraph only accepts attributes stored in tensors (with numerical contents). Consequently, an attribute of all the nodes or edges must have the same shape. In the context of deep learning, those attributes are often called features.

You can assign and retrieve node and edge features via **ndata** and **edata** interface.

In [9]:
# Assign a 3-dimensional node feature vector for each node.
g.ndata['x'] = torch.randn(g.number_of_nodes(), 3)
# Assign a 4-dimensional edge feature vector for each edge.
g.edata['a'] = torch.randn(g.number_of_edges(), 4)
# Assign a 5x4 node feature matrix for each node.  Node and edge features in DGL can be multi-dimensional.
g.ndata['y'] = torch.randn(g.number_of_nodes(), g.number_of_edges(), 4)

print(g.edata['a'])
print(g.num_nodes())
print(g.num_edges())
# Out degrees of the center node
print(g.out_degrees(0))
# In degrees of the center node - note that the graph is directed so the in degree should be 0.
print(g.in_degrees(0))

tensor([[-0.5577, -0.1411, -0.5917,  0.6853],
        [-0.1955, -0.0396,  0.6488,  0.7896],
        [ 1.9263,  1.2331, -0.1440,  0.6483],
        [-0.3525,  1.2251, -0.7027, -0.5663],
        [ 1.6161,  1.3308, -0.5314, -0.6052]])
6
5
5
0


# Graph Transformations
DGL provides many APIs to transform a graph to another such as extracting a subgraph:

In [10]:
# Induce a subgraph from node 0, node 1 and node 3 from the original graph.
sg1 = g.subgraph([0, 1, 3])
# Induce a subgraph from edge 0, edge 1 and edge 3 from the original graph.
sg2 = g.edge_subgraph([0, 1, 3])

You can obtain the node/edge mapping from the subgraph to the original graph by looking into the node feature `dgl.NID`or edge feature `dgl.EID` in the new graph.

In [11]:
# The original IDs of each node in sg1
print(sg1.ndata[dgl.NID])
# The original IDs of each edge in sg1
print(sg1.edata[dgl.EID])
# The original IDs of each node in sg2
print(sg2.ndata[dgl.NID])
# The original IDs of each edge in sg2
print(sg2.edata[dgl.EID])

tensor([0, 1, 3])
tensor([0, 2])
tensor([0, 1, 2, 4])
tensor([0, 1, 3])


`subgraph` and `edge_subgraph` also copies the original features to the subgraph:

In [12]:
# The original node feature of each node in sg1
print(sg1.ndata['x'])
# The original edge feature of each node in sg1
print(sg1.edata['a'])
# The original node feature of each node in sg2
print(sg2.ndata['x'])
# The original edge feature of each node in sg2
print(sg2.edata['a'])

tensor([[-1.2430,  0.0688,  0.5729],
        [ 0.0489, -0.9491, -0.1387],
        [ 0.6180, -0.3271, -0.3056]])
tensor([[-0.5577, -0.1411, -0.5917,  0.6853],
        [ 1.9263,  1.2331, -0.1440,  0.6483]])
tensor([[-1.2430,  0.0688,  0.5729],
        [ 0.0489, -0.9491, -0.1387],
        [-0.0644,  0.7515, -1.5226],
        [-1.4126, -2.5519,  1.5256]])
tensor([[-0.5577, -0.1411, -0.5917,  0.6853],
        [-0.1955, -0.0396,  0.6488,  0.7896],
        [-0.3525,  1.2251, -0.7027, -0.5663]])


Another common transformation is to add a reverse edge for each edge in the original graph with `dgl.add_reverse_edges`. If you have an undirected graph, it is better to convert it into a bidirectional graph first via adding reverse edges.

In [13]:
newg = dgl.add_reverse_edges(g)
newg.edges()

(tensor([0, 0, 0, 0, 0, 1, 2, 3, 4, 5]),
 tensor([1, 2, 3, 4, 5, 0, 0, 0, 0, 0]))

# Loading and Saving Graphs
You can save a graph or a list of graphs via `dgl.save_graphs` and load them back with `dgl.load_graphs`.

In [14]:
# Save graphs
dgl.save_graphs('graph.dgl', g)
dgl.save_graphs('graphs.dgl', [g, sg1, sg2])

# Load graphs
(g,), _ = dgl.load_graphs('graph.dgl')
print(g)
(g, sg1, sg2), _ = dgl.load_graphs('graphs.dgl')
print(g)
print(sg1)
print(sg2)

Graph(num_nodes=6, num_edges=5,
      ndata_schemes={'y': Scheme(shape=(5, 4), dtype=torch.float32), 'x': Scheme(shape=(3,), dtype=torch.float32)}
      edata_schemes={'a': Scheme(shape=(4,), dtype=torch.float32)})
Graph(num_nodes=6, num_edges=5,
      ndata_schemes={'y': Scheme(shape=(5, 4), dtype=torch.float32), 'x': Scheme(shape=(3,), dtype=torch.float32)}
      edata_schemes={'a': Scheme(shape=(4,), dtype=torch.float32)})
Graph(num_nodes=3, num_edges=2,
      ndata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64), 'y': Scheme(shape=(5, 4), dtype=torch.float32), 'x': Scheme(shape=(3,), dtype=torch.float32)}
      edata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64), 'a': Scheme(shape=(4,), dtype=torch.float32)})
Graph(num_nodes=4, num_edges=3,
      ndata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64), 'y': Scheme(shape=(5, 4), dtype=torch.float32), 'x': Scheme(shape=(3,), dtype=torch.float32)}
      edata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64), 'a': Scheme